In [1]:
!pip install pandas

In [2]:
!pip install pyarrow

In [3]:
!pip install scikit-learn

In [4]:
import pandas as pd

In [27]:
# import pyarrow.parquet as pq

df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
# df = pd.read_parquet(, engine='pyarrow')

In [28]:
len(df.columns)

19

In [7]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60


In [8]:
df[df.tpep_dropoff_datetime < df.tpep_pickup_datetime]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
1839180,1,2023-01-20 13:35:00,2023-01-20 13:05:48,1.0,0.8,99.0,N,227,14,1,16.5,0.0,0.5,0.0,0.0,1.0,18.0,0.0,0.0,-29.200000
2124414,1,2023-01-23 10:43:58,2023-01-23 10:29:26,1.0,2.4,99.0,N,102,82,1,21.5,0.0,0.5,0.0,0.0,1.0,23.0,0.0,0.0,-14.533333
2218504,1,2023-01-24 11:00:00,2023-01-24 10:40:48,1.0,0.4,99.0,N,247,169,1,15.5,0.0,0.5,0.0,0.0,1.0,17.0,0.0,0.0,-19.200000


In [9]:
pd.set_option('display.float_format', '{:.2f}'.format)
df.duration.describe()

count   3066766.00
mean         15.67
std          42.59
min         -29.20
25%           7.12
50%          11.52
75%          18.30
max       10029.18
Name: duration, dtype: float64

In [10]:
shape = df.shape[0]
df = df[(df.duration<=60) & (df.duration>=1)]


In [11]:
df.shape[0]/shape

0.9812202822125979

In [12]:
df = df[['duration', 'PULocationID', 'DOLocationID']]

In [13]:
from sklearn.feature_extraction import DictVectorizer

In [14]:
# Step 1: Convert the pickup and dropoff location IDs to strings (to avoid label encoding)
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Step 2: Turn the DataFrame into a list of dictionaries, excluding the non-vectorized column
data_dicts = df.loc[:df.shape[0]/10][['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Step 3: Fit the DictVectorizer and transform the data (only for the location ID columns)
vectorizer = DictVectorizer(sparse=False)  # Set sparse=False if you want a dense matrix
feature_matrix = vectorizer.fit_transform(data_dicts)

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X = feature_matrix
y = df.loc[:df.shape[0]/10]['duration']

model = LinearRegression()
model.fit(X, y)


LinearRegression()

In [19]:

# Predict on training data
y_pred = model.predict(X)

# Calculate RMSE
rmse = mean_squared_error(y, y_pred, squared=False)
print(f"RMSE on train: {rmse:.3f}")

RMSE on train: 7.733


In [20]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')



In [21]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df[df.tpep_dropoff_datetime < df.tpep_pickup_datetime]
df = df[(df.duration<=60) & (df.duration>=1)]
df = df[['duration', 'PULocationID', 'DOLocationID']]



In [22]:
# Convert location IDs to strings for clearer dummy column names
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

In [23]:
# Apply one-hot encoding separately to pickup and dropoff location IDs
#pu_dummies = pd.get_dummies(df['PULocationID'], prefix='PU')
#do_dummies = pd.get_dummies(df['DOLocationID'], prefix='DO')

# Combine original 'duration' with the encoded features
#df_encoded = pd.concat([df[['duration']], pu_dummies, do_dummies], axis=1)

# Show the final encoded DataFrame
#print(df_encoded.shape)

In [24]:
# Step 2: Turn the DataFrame into a list of dictionaries, excluding the non-vectorized column
data_dicts = df.loc[:df.shape[0]/5][['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Step 3: Fit the DictVectorizer and transform the data (only for the location ID columns)
feature_matrix = vectorizer.transform(data_dicts)

In [25]:
X = feature_matrix
y = df.loc[:df.shape[0]/5]['duration']

y_pred = model.predict(X)

# Calculate RMSE
rmse = mean_squared_error(y, y_pred, squared=False)
print(f"RMSE on test: {rmse:.3f}")

RMSE on test: 1502778648.604
